In [191]:
import pandas as pd
from lxml import etree
import requests
import json

---- we can search for a place and then return first hit for a search id ----

In [192]:
#use dict to search for listings
url = "https://www.hemnet.se/locations/show"
params= {
    'q':'Södermalm, Stockholm'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.text



'[{"id":898472,"name":"Södermalm","location_type":"district","slug":"sodermalm","parent_location":{"id":18031,"name":"Stockholms kommun","location_type":"municipality","slug":"stockholms-kommun"}},{"id":476044,"name":"Södermalmstorg","location_type":"street","slug":"sodermalmstorg","parent_location":{"id":18057,"name":"Stockholm","location_type":"postal_city","slug":"stockholm"}},{"id":476225,"name":"Södermalmsallén","location_type":"street","slug":"sodermalmsallen","parent_location":{"id":18057,"name":"Stockholm","location_type":"postal_city","slug":"stockholm"}}]'

In [193]:
json_data=json.loads(response.text)
search_id=json_data[0]['id']

In [194]:
#or make dict with places to search for

#location_ids={'Ringvägen':475989,'Huvudsta':473498}
#search_id=location_ids['Ringvägen']

In [195]:
#use dict to search for listings
url = "https://www.hemnet.se/bostader"
params= {
    'housing_form_groups':'apartments',
    'location_ids':search_id,
    'item_types':'bostadsratt',
    'rooms_min':2,
    'living_area_min':35,
    'new_construction':'exclude'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.url

'https://www.hemnet.se/bostader?housing_form_groups=apartments&location_ids=898472&item_types=bostadsratt&rooms_min=2&living_area_min=35&new_construction=exclude'

--------- w beautiful soup ------------

In [196]:
from bs4 import BeautifulSoup

In [197]:
soup = BeautifulSoup(response.content, "html.parser")

In [198]:
#find div with good data to use
map_results=soup.find(id='results-map')
initial_data=map_results.attrs['data-initial-data']

In [199]:
#find our search key to use
json_data=json.loads(initial_data)
json_data['search_key']


'0179ffb235820946a5e7c087d8f9366c6f8f283d'

In [200]:
url = "https://www.hemnet.se/bostader/search/"+json_data['search_key']

params= {}

payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)

#print(response.text)

In [201]:
r_json = response.json()
r_prop = r_json['properties']
#r_prop

In [206]:
#this output should prob be filtered some to drop crappy columns
df=pd.json_normalize(r_prop, max_level=1)
df.head(3)

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,...,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,broker_name,broker_phone,broker_email,price_change
0,19226978,"[59.31636376245509, 18.064605327405715]",Maria Prästgårdsgata 14,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"{'living_area': '62', 'supplemental_area': '21...",19226978,common_listing,basic,active,for_sale,NaN,NaN,NaN,NaN
1,19226820,"[59.3128281, 18.09094]",Skånegatan 113,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"{'living_area': '57 m²', 'supplemental_area': ...",19226820,common_listing,basic,active,for_sale,NaN,NaN,NaN,NaN
2,19226701,"[59.32030685531627, 18.063923236266938]",Skolgränd 1,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"{'living_area': '44 m²', 'supplemental_area': ...",19226701,common_listing,basic,active,for_sale,NaN,NaN,NaN,NaN


In [203]:
#call graphql directly from listing with ids - example with one id only but can set into a loop
ql_id=df.iloc[0]['id']
ql_id

19226978

In [204]:
url_ql = "https://www.hemnet.se/graphql"

#would love to have payload_ql string formatted better but not really sure how without breaking it
payload_ql="{\"query\":\"query imagesForListing($id: ID!) {\\r\\n    listing(id: $id) \\r\\n    {  id  __typename  streetAddress  isSaved  brokerAgency \\r\\n    {    name    id    brokerCustomization \\r\\n    {      compactLogoUrl      largeLogoUrl: logoUrl(format: BROKER_CUSTOMIZATION_LARGE)      __typename    \\r\\n    }    __typename  }  ... on ActivePropertyListing \\r\\n    {    liveStreams(scope: ENDED) \\r\\n    {      embedUrl      __typename    \\r\\n    }    videoAttachment: attachment(type: VIDEO) \\r\\n    {      id      attachmentType      ... on VideoAttachment {     videoHemnetUrl        __typename     }\\r\\n          __typename    \\r\\n    }    threeDAttachment: attachment(type: THREE_D) {\\r\\n              id      ... on ThreeDAttachment {\\r\\n                          url        __typename      \\r\\n                          }      __typename    }\\r\\n                              isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {\\r\\n                                        images {\\r\\n                                                    url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }  ... on ProjectUnit {    liveStreams(scope: ENDED) {      embedUrl      __typename    }    videoAttachment: attachment(type: VIDEO) {      id      attachmentType      ... on VideoAttachment {        videoHemnetUrl        __typename      }      __typename    }    threeDAttachment: attachment(type: THREE_D) {      id      ... on ThreeDAttachment {        url        __typename      }      __typename    }    isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {      images {        url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }\\r\\n    }\\r\\n    }\",\"variables\":{\"id\":"+str(ql_id)+"}}"

headers_ql = {
  'Content-Type': 'application/json',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response_ql = requests.request("POST", url_ql, headers=headers_ql, data=payload_ql)

#print(response_ql.text)
ql_path=response_ql.json()['data']['listing']['images']['images']

ql_df=pd.json_normalize(ql_path)
#ql_df

In [205]:
#print all images from listing item
from IPython.display import Image
for index,row in ql_df.iterrows():
   display(Image(url=row['url'], width=300))